In [ ]:
from dotenv import load_dotenv
import requests 
import json 
from bs4 import BeautifulSoup
import os
import google.generativeai as genai
from bs4 import BeautifulSoup

c:\Users\sadhi\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
load_dotenv()
api = os.environ['API']
search_api = os.environ['API_SEARCH']
search_engine = os.environ['SEARCH_ENGINE_ID']
genai.configure(api_key=api)

In [ ]:
model = genai.GenerativeModel(model_name='gemini-1.5-pro-latest')

In [ ]:
def google_search(query):
    """
    Function to perform Google Search using Custom Search JSON API.
    """
    search_url = "https://www.googleapis.com/customsearch/v1"
    params = {
        'key': search_api,
        'cx': search_engine,
        'q': query,
        'gl': "IN"
    }
    
    response = requests.get(search_url, params=params)
    if response.status_code == 200:
        search_results = response.json()
        links = [item['link'] for item in search_results.get('items', [])]
        return links
    else:
        print(f"Error: {response.status_code}")
        return []

In [ ]:
sample_query = "Indian corporate employees mental health and support"
article_links = google_search(sample_query)
print("Fetched Links:", article_links)

Fetched Links: ['https://pmc.ncbi.nlm.nih.gov/articles/PMC9110774/', 'https://www.mindsharepartners.org/blog/the-indian-community-is-grappling-with-our-mental-health-and-we-desperately-need-help', 'https://journals.lww.com/indianjpsychiatry/fulltext/2024/66002/mental_health_and_well_being_at_the_workplace.16.aspx', 'https://www2.deloitte.com/content/dam/Deloitte/global/Documents/Life-Sciences-Health-Care/gx-mental-health-2022-report-noexp.pdf', 'https://www.csis.org/blogs/new-perspectives-asia/private-sector-solutions-india-mental-health-workplace', 'https://www.lyrahealth.com/', 'https://www.ihs.gov/', 'https://www.doi.gov/pmb/hr/eap', 'https://www.sac-isc.gc.ca/eng/1581971225188/1581971250953', 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6824928/']


In [ ]:
def fetch_article_content(url):
    """
    Function to fetch and extract the main content from a given article URL.
    Adds headers to the request to mimic a browser.
    """
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36",
        "Accept-Language": "en-US,en;q=0.9",
        "Accept-Encoding": "gzip, deflate, br",
        "Connection": "keep-alive",
    }
    
    try:
        # Send a GET request to the article URL with headers
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()  # Check for successful response
        
        # Parse HTML content with BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Extract main text content from <p> tags
        paragraphs = soup.find_all('p')
        article_text = "\n".join([para.get_text() for para in paragraphs if para.get_text()])
        
        return article_text.strip()  # Return clean, extracted text content
    
    except requests.exceptions.RequestException as e:
        print(f"Error fetching article content from {url}: {e}")
        return None

In [ ]:
def summarize_text_with_gemini(text):
    """
    Function to generate a detailed summary of the given text using the Gemini model.
    Emphasizes the abstract, conclusion, and other key points like applications,
    advantages, disadvantages, technical highlights, future directions, and results.
    """
    # Define the pre-prompt structure for a detailed summary
    preprompt = f"""
    Given the following article text, provide a detailed summary that focuses on the following:
    
    1. **Main Focus**: What is the article primarily about? Describe the core subject and objective of the article.
    2. **Key Insights**: Extract any significant insights, findings, or conclusions presented in the article.
    3. **Statistics/Data**: Mainly focus on the STATISTICS NUMBERS PERCENTAGES ETC Identify any key data, statistics, or results presented.
    4. **Conclusion**: Provide a summary of the conclusions or final remarks made by the author.
    
    Additionally, provide an overall summary of the article that encapsulates its main points and purpose.

    Text: 
    Darft the problem with stas in 100 words'{text}'
    """
    
    # Generate content using the model
    response = model.generate_content(preprompt)
    story = response.text
    return story 

In [ ]:
for i in article_links:
    extracted_text = fetch_article_content(i)
    if extracted_text:
        summary = summarize_text_with_gemini(extracted_text)
        print(f"Article URL: {i}")
        print(f"Summary:\n{summary}\n")

Article URL: https://pmc.ncbi.nlm.nih.gov/articles/PMC9110774/
Summary:
**1. Main Focus:**

The article is a scoping review of workplace mental health interventions in India. It aims to identify and summarize the types of mental health programs offered by Indian organizations, their key features, and gaps in existing interventions. The review also explores the prevalence and impact of mental health problems among Indian workers.

**2. Key Insights:**

* Mental health issues are prevalent among Indian workers, impacting their productivity and overall well-being.
* Workplace mental health initiatives in India are growing, but often lack comprehensive needs assessments, impact evaluations, and supporting workplace policies.
* Most interventions focus on curative approaches like counseling, rather than preventive and promotional activities.
* There's a need for more research on risk factors, cost-effectiveness, and best practices for workplace mental health interventions in the Indian cont

ResourceExhausted: 429 Resource has been exhausted (e.g. check quota).